In [72]:
import time
import datetime
import math
import numpy as np
import pandas as pd
import lightgbm as lgb
from dateutil.parser import parse
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error


train = pd.read_csv('./chouchoukan.csv')
dev = pd.read_csv('./chouchoukan_dev.csv')

# train = train.drop(['data_min_week','data_median','data_min','data_max','data_mean'],axis = 1)
# test = test.drop(['data_min_week','data_median','data_min','data_max','data_mean'],axis = 1)
# print(train['holiday'])
# del test['holiday']
# del dev['holiday']
# del train['holiday']
# train = train.astype('float64')
predictors = [f for f in train.columns if f not in ['phone_num','time_stamp','wind_v','wind_u']]

def evalerror(pred, df):
    label = df.get_label().values.copy()
    score = math.sqrt(mean_squared_error(label,pred))
    return ('mse',score,False)

print('Training..')
params = {
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'sub_feature': 0.7,
    'num_leaves': 60,
    'colsample_bytree': 0.7,
    'feature_fraction': 0.7,
    'min_data': 100,
    'min_hessian': 1,
    'verbose': -1,
}

# from sklearn.cross_validation import train_test_split  
# train_feat, val_feat, train_y, val_y = train_test_split(train[predictors],train['phone_num'],test_size = 0.2,random_state=42) 
train_feat = train[predictors]
train_y = train['phone_num']

val_feat = dev[predictors]
val_y = dev['phone_num']
lgb_train = lgb.Dataset(train_feat, train_y,categorical_feature=['loc_hour','loc_week','loc_rate','loc_weather','hour_rate','hour_weather','hour_week','week_rate','week_weather','rate_weather','loc_id','week','hour','weather','rate'])
lgb_val = lgb.Dataset(val_feat, val_y,categorical_feature=['loc_hour','loc_week','loc_rate','loc_weather','hour_rate','hour_weather','hour_week','week_rate','week_weather','rate_weather','loc_id','week','hour','weather','rate'])
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=50000,
                valid_sets=lgb_val,
                verbose_eval=100,
                feval=evalerror,
                early_stopping_rounds=100)
print(pd.Series(gbm.feature_importance(),index = predictors).sort_values(ascending = False))
# import pickle
# with open('gbm_pm25_ld.pkl', 'wb') as f:
#     pickle.dump(gbm, f)





Training..
Training until validation scores don't improve for 100 rounds.


/home/zhangqq/.local/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/zhangqq/.local/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[100]	valid_0's rmse: 191.07	valid_0's mse: 191.07
[200]	valid_0's rmse: 111.668	valid_0's mse: 111.668
[300]	valid_0's rmse: 98.2631	valid_0's mse: 98.2631
[400]	valid_0's rmse: 97.5313	valid_0's mse: 97.5313
Early stopping, best iteration is:
[371]	valid_0's rmse: 97.3411	valid_0's mse: 97.3411
data_mean_week_hour      3517
data_max_week_hour       2916
data_min_week_hour       2078
data_median_week_hour    1397
pressure                  978
hour_rate                 891
NO2                       860
loc_rate                  826
O3_8h                     811
humidity                  657
loc_week                  627
SSD                       622
temperature               541
week_num                  468
PM10                      466
week_rate                 465
AQI                       403
SO2                       368
wind_speed                361
hour_weather              310
loc_weather               298
CO                        288
loc_id                    287
hour_week   

In [73]:
import time
import datetime
import math
import numpy as np
import pandas as pd
import lightgbm as lgb
from dateutil.parser import parse
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error


train = pd.read_csv('./chouchoukan2.csv')

test = pd.read_csv('./test_X.csv')
# train = train.drop(['data_min_week','data_median','data_min','data_max','data_mean'],axis = 1)
# test = test.drop(['data_min_week','data_median','data_min','data_max','data_mean'],axis = 1)
# print(train['holiday'])
# del test['holiday']

# del train['holiday']
# train = train.astype('float64')
predictors = [f for f in train.columns if f not in ['phone_num','time_stamp','wind_v','wind_u']]

def evalerror(pred, df):
    label = df.get_label().values.copy()
    score = math.sqrt(mean_squared_error(label,pred))
    return ('mse',score,False)

print('Training..')
params = {
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'sub_feature': 0.7,
    'num_leaves': 60,
    'colsample_bytree': 0.7,
    'feature_fraction': 0.7,
    'min_data': 100,
    'min_hessian': 1,
    'verbose': -1,
}

# from sklearn.cross_validation import train_test_split  
# train_feat, val_feat, train_y, val_y = train_test_split(train[predictors],train['phone_num'],test_size = 0.2,random_state=42) 
train_feat = train[predictors]
train_y = train['phone_num']

# val_feat = dev[predictors]
# val_y = dev['phone_num']
lgb_train = lgb.Dataset(train_feat, train_y,categorical_feature=['loc_hour','loc_week','loc_rate','loc_weather','hour_rate','hour_weather','hour_week','week_rate','week_weather','rate_weather','loc_id','week','hour','weather','rate'])
# lgb_val = lgb.Dataset(val_feat, val_y,categorical_feature=['loc_hour','loc_week','loc_rate','loc_weather','hour_rate','hour_weather','hour_week','week_rate','week_weather','rate_weather','loc_id','week','hour','weather','rate'])
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=371,
                valid_sets=lgb_train,
                verbose_eval=100,
                feval=evalerror,
                )
print(pd.Series(gbm.feature_importance(),index = predictors).sort_values(ascending = False))
# import pickle
# with open('gbm_pm25_ld.pkl', 'wb') as f:
#     pickle.dump(gbm, f)
test_x = test[['loc_id','time_stamp']]


test_x['time_stamp'] = test_x['time_stamp'].apply(lambda x: x.split(':')[0])  


test_y = gbm.predict(test[predictors])




Training..


/home/zhangqq/.local/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	training's rmse: 208.35	training's mse: 208.35
[200]	training's rmse: 127.519	training's mse: 127.519
[300]	training's rmse: 104.386	training's mse: 104.386
data_mean_week_hour      3592
data_max_week_hour       1898
data_median_week_hour    1383
loc_rate                 1361
data_min_week_hour       1279
loc_week                 1271
O3_8h                    1079
hour_rate                1051
week_num                  907
week_rate                 823
NO2                       664
humidity                  589
pressure                  587
SSD                       557
temperature               546
PM10                      530
loc_hour                  528
PM2.5                     491
loc_weather               425
AQI                       351
hour_week                 318
SO2                       280
hour_weather              262
CO                        258
wind_speed                251
loc_id                    217
week_weather              144
hour                      1

/home/zhangqq/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [74]:
test_y = pd.DataFrame(test_y)
test_y.columns = ['num_of_people']
test_y['num_of_people'] = test_y['num_of_people'].astype('int32')
data = pd.concat([test_x, test_y],axis = 1)



data['num_of_people'][data['num_of_people']<=0] = 20
data.to_csv('submit.csv',index = False)

/home/zhangqq/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [75]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline


def mkdir(path):
    isExists=os.path.exists(path)
    if not isExists:
        os.makedirs(path) 


path_to_bj_aq = "./data/month456/test/"

bj_csv_list  = os.listdir(path_to_bj_aq)

for csv in bj_csv_list :
    if csv != '.DS_Store' and not csv.startswith("._"):
        path_to_file = path_to_bj_aq + csv
        
        data = pd.read_csv(path_to_file)
        i = csv.split('.')[0]

        data['time_stamp'] = pd.to_datetime(data['time_stamp'])
        data = data.groupby(['time_stamp','loc_id'],as_index = False).count()
        data['date'] = data['time_stamp'].apply(lambda x:str(x)[:10])
        data['hour'] = data['time_stamp'].apply(lambda x:int(str(x)[11:13]))
        data['week'] = data['time_stamp'].apply(lambda x:x.weekday())

        for j in range(1,34):
            data_part = data[data['loc_id'] == j]
            mkdir('./datatest/station_%s' % (j))
            data_part.to_csv('datatest/station_%s/month_%s.csv' % (j,i),index = False)
            
            
            
#数据预处理


# data_meo = pd.read_csv('data/data_meo.csv')   
datas = []
for i in range(1,34):
    path_to_bj_aq = "./datatest/station_"+str(i)+"/"

    bj_csv_list  = os.listdir(path_to_bj_aq)
    
    for csv in bj_csv_list :
        if csv != '.DS_Store' and not csv.startswith("._") :
            
            path_to_file = path_to_bj_aq + csv
            data = pd.read_csv(path_to_file)  
            if data.empty == False:
                datas.append(data)

data_all = pd.concat(datas, ignore_index=True)


# data_all = pd.merge(data_all, data_meo, how='left', on=['time_stamp'])
data_all.rename(index=str, columns={"phone_id": "num_of_people"},inplace = True)

data_all = data_all[['loc_id','time_stamp',"num_of_people"]]


data_all['time_stamp'] = data_all['time_stamp'].apply(lambda x: x.split(':')[0])  



data_all.to_csv('submit0.csv',index = False)

In [76]:
train = pd.read_csv('./submit.csv')
dev = pd.read_csv('./submit0.csv')
data = pd.merge(train, dev, how='inner', on=['time_stamp','loc_id'])

score = math.sqrt(mean_squared_error(data['num_of_people_x'],data['num_of_people_y']))
print(score)

157.20776240937306
